In [28]:
# Dependencies
import pandas as pd
import requests
import json
import time
import sqlite3 as db

# Ignore SQLITE warnings related to Decimal numbers in the Chinook database.
import warnings
warnings.filterwarnings('ignore')

# Import Dependencies for SQLALCHEMY
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
from sqlalchemy import func
from sqlalchemy import create_engine, ForeignKey, MetaData, Table, select
from sqlalchemy import Column, Date, Integer, String, Numeric, Float

from sqlalchemy.ext.declarative import declarative_base
import pandas as pd
from sqlalchemy.ext.declarative import declarative_base

# Create an engine for the sqlite database
engine = create_engine("sqlite:///data/climate_change.sqlite", echo=False)
conn = engine.connect()


# EIA developer API key
from config import api_key

# Build the endpoint URL
# http://api.eia.gov/series/?api_key=YOUR_API_KEY_HERE&series_id=EMISS.CO2-TOTV-TT-PE-MD.A
#target_url = 'http://api.eia.gov/category/?api_key=3f7c6e0096672dc16b49796f3a89c943&category_id=240690'
url='http://api.eia.gov/category/?'
id='2251670'
query_url = url + "api_key=" + api_key + "&category_id=" + id

In [29]:
# Run a request to endpoint and convert result to json
co2_data = requests.get(query_url).json()

# Print the json
print(json.dumps(co2_data, indent=4, sort_keys=True))

{
    "category": {
        "category_id": "2251670",
        "childcategories": [
            {
                "category_id": 2251671,
                "name": "Maryland"
            },
            {
                "category_id": 2251672,
                "name": "Alabama"
            },
            {
                "category_id": 2251673,
                "name": "Alaska"
            },
            {
                "category_id": 2251674,
                "name": "Arizona"
            },
            {
                "category_id": 2251675,
                "name": "Arkansas"
            },
            {
                "category_id": 2251676,
                "name": "California"
            },
            {
                "category_id": 2251677,
                "name": "Colorado"
            },
            {
                "category_id": 2251678,
                "name": "Connecticut"
            },
            {
                "category_id": 2251679,
                "name": "Delaw

In [30]:
# Count No of state and get the chield category_id
state_count=(co2_data["category"]["childcategories"])
print (len(state_count))
print(co2_data["category"]["childcategories"][0]['category_id'])

52
2251671


In [31]:
# Declare a place holder array to append category_id of each state 
state_id=[]
i=0
while i < len(state_count) :
    state_id.append(co2_data["category"]["childcategories"][i]['category_id'])
    i+=1

In [32]:
state_id[44]

2251715

In [33]:
len(state_id)

52

In [34]:
# Declare a place holder array to append serise_id of each state(Range 0 to 43 and 45 to end of array)
# Note: The 44th category id 2251715 ("name": "United States") has unknown error and excluded intentionally 
           
serise_id=[]

url='http://api.eia.gov/category/?'
#id="2251671"
i=0
#j=0
while i < 44 :    
    id=str(state_id[i])
    query_url = url + "api_key=" + api_key + "&category_id=" + id    
    co2_data = requests.get(query_url).json()    
    serise_id.append(co2_data["category"]["childseries"][59]["series_id"])   
    i+=1

In [35]:
i=45
while i < len(state_id) :    
    id=str(state_id[i])
    query_url = url + "api_key=" + api_key + "&category_id=" + id    
    co2_data = requests.get(query_url).json()    
    serise_id.append(co2_data["category"]["childseries"][59]["series_id"])    
    i+=1

In [36]:
serise_id

['EMISS.CO2-TOTV-TT-TO-MD.A',
 'EMISS.CO2-TOTV-TT-TO-AL.A',
 'EMISS.CO2-TOTV-TT-TO-AK.A',
 'EMISS.CO2-TOTV-TT-TO-AZ.A',
 'EMISS.CO2-TOTV-TT-TO-AR.A',
 'EMISS.CO2-TOTV-TT-TO-CA.A',
 'EMISS.CO2-TOTV-TT-TO-CO.A',
 'EMISS.CO2-TOTV-TT-TO-CT.A',
 'EMISS.CO2-TOTV-TT-TO-DE.A',
 'EMISS.CO2-TOTV-TT-TO-DC.A',
 'EMISS.CO2-TOTV-TT-TO-FL.A',
 'EMISS.CO2-TOTV-TT-TO-GA.A',
 'EMISS.CO2-TOTV-TT-TO-HI.A',
 'EMISS.CO2-TOTV-TT-TO-ID.A',
 'EMISS.CO2-TOTV-TT-TO-IL.A',
 'EMISS.CO2-TOTV-TT-TO-IN.A',
 'EMISS.CO2-TOTV-TT-TO-IA.A',
 'EMISS.CO2-TOTV-TT-TO-KS.A',
 'EMISS.CO2-TOTV-TT-TO-KY.A',
 'EMISS.CO2-TOTV-TT-TO-LA.A',
 'EMISS.CO2-TOTV-TT-TO-ME.A',
 'EMISS.CO2-TOTV-TT-TO-MA.A',
 'EMISS.CO2-TOTV-TT-TO-MI.A',
 'EMISS.CO2-TOTV-TT-TO-MN.A',
 'EMISS.CO2-TOTV-TT-TO-MS.A',
 'EMISS.CO2-TOTV-TT-TO-MO.A',
 'EMISS.CO2-TOTV-TT-TO-MT.A',
 'EMISS.CO2-TOTV-TT-TO-NE.A',
 'EMISS.CO2-TOTV-TT-TO-NV.A',
 'EMISS.CO2-TOTV-TT-TO-NH.A',
 'EMISS.CO2-TOTV-TT-TO-NJ.A',
 'EMISS.CO2-TOTV-TT-TO-NM.A',
 'EMISS.CO2-TOTV-TT-TO-NY.A',
 'EMISS.CO

In [37]:
len(serise_id)

51

In [38]:
serise_id[6]

'EMISS.CO2-TOTV-TT-TO-CO.A'

In [39]:
# Create an Empty DataFrame with col Year CO2DATA and State 
df=pd.DataFrame(columns=['Year', 'CO2DATA', 'State'])
# Create API call and get data and accumulate on df
url='https://api.eia.gov/series/?'
for x in serise_id:      
    sid=x
    # Create query url with serise id
    query_url = url + "api_key=" + api_key + "&series_id=" + sid

    # Run a request to endpoint and convert result to json
    co2_data = requests.get(query_url).json()

    # Get the state name
    stname=co2_data["series"][0]['name']    
    nstate=stname.split(',',1)[1]
    stateName=nstate.split(',',1)[1].lstrip() 
    ydata=co2_data["series"][0]['data']
    # Create new df with data 
    co2df=pd.DataFrame(ydata,columns=['Year','CO2DATA'])
    # Add state name as column to df
    co2df['State']=stateName
    # Append df
    df=df.append(co2df)

In [40]:
#df=df.reset_index(drop=True)
#df.to_csv('all_state_co2.csv')
#df

In [41]:
df.head()

,Year,CO2DATA,State
0,2016,57.557961,Maryland
1,2015,59.473748,Maryland
2,2014,61.391691,Maryland
3,2013,59.152914,Maryland
4,2012,59.859472,Maryland


In [42]:
# Create a database with table model or schema
Base = declarative_base()

class co2(Base):

    __tablename__ = "co"

    ID = Column(Integer, primary_key=True)       
    Year = Column(Integer)
    CO2DATA = Column(Float)
    State = Column(String) 

    def __init__(self, name):
        self.name = name  

Base.metadata.create_all(engine)

In [43]:
# Query all database and make sure the new one created
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
print(Base.classes.keys())
co = Base.classes.co

['co']


In [44]:
# Create a database session object
session = Session(engine)
tableToWriteTo = 'co'

In [45]:
listToWrite = df.to_dict(orient='records')
listToWrite

[{'Year': '2016', 'CO2DATA': 57.557961, 'State': 'Maryland'},
 {'Year': '2015', 'CO2DATA': 59.473748, 'State': 'Maryland'},
 {'Year': '2014', 'CO2DATA': 61.391691, 'State': 'Maryland'},
 {'Year': '2013', 'CO2DATA': 59.152914, 'State': 'Maryland'},
 {'Year': '2012', 'CO2DATA': 59.859472, 'State': 'Maryland'},
 {'Year': '2011', 'CO2DATA': 64.432162, 'State': 'Maryland'},
 {'Year': '2010', 'CO2DATA': 69.124429, 'State': 'Maryland'},
 {'Year': '2009', 'CO2DATA': 70.482085, 'State': 'Maryland'},
 {'Year': '2008', 'CO2DATA': 73.767925, 'State': 'Maryland'},
 {'Year': '2007', 'CO2DATA': 77.533365, 'State': 'Maryland'},
 {'Year': '2006', 'CO2DATA': 77.166737, 'State': 'Maryland'},
 {'Year': '2005', 'CO2DATA': 82.005117, 'State': 'Maryland'},
 {'Year': '2004', 'CO2DATA': 79.851084, 'State': 'Maryland'},
 {'Year': '2003', 'CO2DATA': 79.299126, 'State': 'Maryland'},
 {'Year': '2002', 'CO2DATA': 76.48787, 'State': 'Maryland'},
 {'Year': '2001', 'CO2DATA': 76.532899, 'State': 'Maryland'},
 {'Year':

In [46]:
metadata = sqlalchemy.schema.MetaData(bind=engine,reflect=True)
table = sqlalchemy.Table(tableToWriteTo, metadata, autoload=True)

In [47]:
conn.execute(table.insert(), listToWrite)

In [48]:
session.commit()

In [49]:
# display all tables
Base.classes.keys()

['co']

In [50]:
co = Base.classes.co

In [51]:
t = Table("co", metadata, autoload=True)
columns = [m.key for m in t.columns]
columns

['ID', 'Year', 'CO2DATA', 'State']

In [52]:
# List all of the data found in the invoices table 
session.query(co.ID,co.State, co.Year, co.CO2DATA).all()

[(1, 'Maryland', 2016, 57.557961),
 (2, 'Maryland', 2015, 59.473748),
 (3, 'Maryland', 2014, 61.391691),
 (4, 'Maryland', 2013, 59.152914),
 (5, 'Maryland', 2012, 59.859472),
 (6, 'Maryland', 2011, 64.432162),
 (7, 'Maryland', 2010, 69.124429),
 (8, 'Maryland', 2009, 70.482085),
 (9, 'Maryland', 2008, 73.767925),
 (10, 'Maryland', 2007, 77.533365),
 (11, 'Maryland', 2006, 77.166737),
 (12, 'Maryland', 2005, 82.005117),
 (13, 'Maryland', 2004, 79.851084),
 (14, 'Maryland', 2003, 79.299126),
 (15, 'Maryland', 2002, 76.48787),
 (16, 'Maryland', 2001, 76.532899),
 (17, 'Maryland', 2000, 76.780629),
 (18, 'Maryland', 1999, 76.783697),
 (19, 'Maryland', 1998, 74.368834),
 (20, 'Maryland', 1997, 71.775523),
 (21, 'Maryland', 1996, 71.377163),
 (22, 'Maryland', 1995, 69.305464),
 (23, 'Maryland', 1994, 69.840096),
 (24, 'Maryland', 1993, 68.832021),
 (25, 'Maryland', 1992, 66.242117),
 (26, 'Maryland', 1991, 68.521645),
 (27, 'Maryland', 1990, 69.745026),
 (28, 'Maryland', 1989, 76.737665),
 (